In [1]:
import numpy as np
import torch
from torch import nn, optim
import torch.nn.functional as F

import sys
sys.path.append("..") 
import d2lzh_pytorch as d2l
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

import zipfile
def load_data_jay_lyrics():
    """加载周杰伦歌词数据集"""
    with zipfile.ZipFile('data/jaychou_lyrics.txt.zip') as zin:
        with zin.open('jaychou_lyrics.txt') as f:
            corpus_chars = f.read().decode('utf-8')
    corpus_chars = corpus_chars.replace('\n', ' ').replace('\r', ' ')
    corpus_chars = corpus_chars[0:10000]
    idx_to_char = list(set(corpus_chars))
    char_to_idx = dict([(char, i) for i, char in enumerate(idx_to_char)])
    vocab_size = len(char_to_idx)
    corpus_indices = [char_to_idx[char] for char in corpus_chars]
    return corpus_indices, char_to_idx, idx_to_char, vocab_size

(corpus_indices, char_to_idx, idx_to_char, vocab_size) = load_data_jay_lyrics()

# 初始化模型参数

In [2]:
num_inputs, num_hiddens, num_outputs = vocab_size, 256, vocab_size
print('will use', device)

def get_params():
    def _one(shape):
        ts = torch.tensor(np.random.normal(0, 0.01, size=shape), device=device, dtype=torch.float32)
        return torch.nn.Parameter(ts, requires_grad=True)
    def _three():
        return (_one((num_inputs, num_hiddens)),
                _one((num_hiddens, num_hiddens)),
                torch.nn.Parameter(torch.zeros(num_hiddens, device=device, dtype=torch.float32), requires_grad=True))

    W_xi, W_hi, b_i = _three()  # 输入门参数
    W_xf, W_hf, b_f = _three()  # 遗忘门参数
    W_xo, W_ho, b_o = _three()  # 出门参数
    W_xc, W_hc, b_c = _three()  # 候选记忆细胞参数

    # 输出层参数
    W_hq = _one((num_hiddens, num_outputs))
    b_q = torch.nn.Parameter(torch.zeros(num_outputs, device=device, dtype=torch.float32), requires_grad=True)
    return nn.ParameterList([W_xi, W_hi, b_i, W_xf, W_hf, b_f, W_xo, W_ho, b_o, W_xc, W_hc, b_c, W_hq, b_q])


will use cpu


# 定义模型

In [3]:
def init_lstm_state(batch_size, num_hiddens, device):
    # 2个初始状态
    return (
        torch.zeros((batch_size, num_hiddens), device=device), # 初始隐藏状态
        torch.zeros((batch_size, num_hiddens), device=device)  # 初始记忆细胞
    )


In [4]:
def lstm(inputs, state, params):
    [W_xi, W_hi, b_i, W_xf, W_hf, b_f, W_xo, W_ho, b_o, W_xc, W_hc, b_c, W_hq, b_q] = params
    (H,C) = state
    outputs = []
    for X in inputs:
        I = torch.sigmoid(torch.matmul(X, W_xi) + torch.matmul(H, W_hi) + b_i)
        F = torch.sigmoid(torch.matmul(X, W_xf) + torch.matmul(H, W_hf) + b_f)
        O = torch.sigmoid(torch.matmul(X, W_xo) + torch.matmul(H, W_ho) + b_o)
        C_tilda = torch.tanh(torch.matmul(X, W_xc) + torch.matmul(H, W_hc) + b_c)
        C = F * C + I * C_tilda
        H = O * C.tanh()
        Y = torch.matmul(H, W_hq) + b_q
        outputs.append(Y)
    return outputs, (H, C)

In [5]:
num_epochs, num_steps, batch_size, lr, clipping_theta = 160, 35, 32, 1e2, 1e-2
pred_period, pred_len, prefixes = 40, 50, ['分开', '不分开']

d2l.train_and_predict_rnn(lstm, get_params, init_lstm_state, num_hiddens,
                          vocab_size, device, corpus_indices, idx_to_char,
                          char_to_idx, False, num_epochs, num_steps, lr,
                          clipping_theta, batch_size, pred_period, pred_len,
                          prefixes)


epoch 40, perplexity 215.012684, time 1.44 sec
 - 分开 我不不我 我不你我 我不你我 我不你我 我不你我 我不你我 我不你我 我不你我 我不你我 我不你我
 - 不分开 我不不我 我不你我 我不你我 我不你我 我不你我 我不你我 我不你我 我不你我 我不你我 我不你我
epoch 80, perplexity 66.258172, time 1.51 sec
 - 分开 我想你的爱我 你不我 你你的我 我不要 我不要 我不要 我不要 我不要 我不了 我不了 我不了 我
 - 不分开 我不你的爱我 你要我 你你的我 我不要 我不要 我不要 我不要 我不要 我不了 我不了 我不了 我
epoch 120, perplexity 14.805745, time 1.78 sec
 - 分开 我想你 你爱我 想想好好 我不好这样 我的话好生活 快知后觉 我该好好节活 我知好觉 我该好好节活
 - 不分开 我爱你 你爱我 想想好好 我不好好活 我的好好 我该好好生活 我不好好生活 不知不觉 又你了离开我
epoch 160, perplexity 3.982724, time 1.59 sec
 - 分开 我想带你你经很久 想想想 说你眼睛看着我 别发抖 快给我抬起头 有话去对医药箱说 别怪我 别怪我 
 - 不分开 我已经这生经 后知后觉 又过了一个秋 后知后觉 我该好好生活 我该好好生活 静知不觉 你已经离开我


# 简洁实现

In [6]:
lr = 1e-2
lstm_layer = nn.LSTM(input_size=vocab_size,hidden_size=num_hiddens)
model = d2l.RNNModel(lstm_layer,vocab_size)
d2l.train_and_predict_rnn_pytorch(model, num_hiddens, vocab_size, device,
                                corpus_indices, idx_to_char, char_to_idx,
                                num_epochs, num_steps, lr, clipping_theta,
                                batch_size, pred_period, pred_len, prefixes)

epoch 40, perplexity 1.026819, time 1.17 sec
 - 分开始共渡每一天 手牵手 一步两步三步四步望著天 看星星 一颗两颗三颗四颗 连成线背著背默默许下心愿 看
 - 不分开 我想就这样牵着你的手不放开 爱可不可以简简单单没有伤害 你 靠着我的肩膀 你 在我胸口睡著 像这样
epoch 80, perplexity 1.012411, time 1.22 sec
 - 分开 我想就这样牵着你的手不放开 爱可不可以简简单单没有伤害 你 靠着我的肩膀 你 在我胸口睡著 像这样
 - 不分开 我想就这样牵着你的手不放开 爱可不可以简简单单没有伤害 你 靠着我的肩膀 你 在我胸口睡著 像这样
epoch 120, perplexity 1.027229, time 1.17 sec
 - 分开 我被我拖着你 我不开 爱情来的太快就像龙卷风 离不开暴风圈来不及逃 我不能再想 我不能再想 我不 
 - 不分开 我变得很主动 若爱上一个人 什么都会值得去做 我想大声宣布 对你依依不舍 连隔壁邻居都猜到我现在的
epoch 160, perplexity 1.009419, time 1.25 sec
 - 分开 我想就这样牵着你的手不放开 爱能不能够永远单纯没有悲哀 我 想带你骑单车 我 想和你看棒球 想这样
 - 不分开 我说不起 什么 我变得很主动 若爱上一个人 什么都会值得去做 我想大声宣布 对你依依不舍 连隔壁邻
